# Diversity in Recommendation Systems - Bounded Greedy Algorithm

## Reading the Data

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
from scipy.spatial.distance import pdist
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv('../datasets/vectors.tsv', sep='\t', header=None)
df_labels = pd.read_csv('../datasets/metadata.tsv', sep='\t', names=['Labels'])

In [ ]:
df['vector'] = df[:].values.tolist()
dfnew = pd.concat([df_labels, df], axis = 1)
dfnew.head()

,Labels,0,1,2,3,4,5,6,7,8,...,119,120,121,122,123,124,125,126,127,vector
0,------Username------,0.033573,-0.003060,0.028717,-0.019389,-0.015540,0.025682,-0.044330,-0.046595,0.041823,...,-0.013637,-0.001407,0.037057,0.010626,-0.008644,0.045264,-0.003153,0.020575,-0.005486,"[0.033573065, -0.003059756, 0.0287168729999999..."
1,----Michel----,0.044829,-0.012990,0.007363,-0.047944,0.034858,-0.017518,-0.002088,0.023707,-0.005278,...,0.026392,-0.028762,-0.010561,-0.047328,0.031681,0.005442,0.046587,0.007106,-0.039871,"[0.04482906, -0.0129903555, 0.0073633566, -0.0..."
2,----The_Truth-----,-0.018356,0.031759,-0.040295,0.005679,0.026772,-0.031120,0.001290,0.013565,0.048065,...,0.008656,0.045598,-0.014290,0.046919,0.028597,-0.021231,-0.026456,0.010762,-0.026287,"[-0.018355988, 0.03175925, -0.04029547, 0.0056..."
3,----meh----,0.024258,0.047903,-0.024987,-0.042511,-0.026175,0.010981,0.045995,0.024367,0.005051,...,-0.021706,-0.036309,0.040087,0.015059,-0.002017,0.029141,0.031405,-0.049530,-0.011700,"[0.024258208, 0.047902945, -0.02498666, -0.042..."
4,----petrichor----,0.006294,0.022231,0.015397,0.046233,0.042291,-0.029548,0.029260,-0.014885,-0.006767,...,0.032619,0.000161,0.010016,0.019834,-0.048005,0.046414,-0.047809,-0.028996,0.044578,"[0.006293676999999999, 0.02223121, 0.015396725..."


## Recommending the Top 50 Similar Subreddits

Here, I am using the same method defined in the previous notebook, but instead recommending the 50 most similar subreddits rather than only 10.

In [ ]:
def get_similar_subreddits(sub_name, num_subs_to_reccomend):
    similarities = []
    sub_name_vector = dfnew['vector'][dfnew['Labels'] == sub_name].to_numpy()[0]
    sub_name_vector = np.array(sub_name_vector).reshape(1, -1)
    for vector in dfnew['vector'].tolist():
        vector = np.array(vector).reshape(1, -1)
        similarities.append(cosine_similarity(sub_name_vector, vector))

    pairs = list(zip(dfnew['Labels'], similarities))
    closest_subs = sorted(pairs, key=lambda item: item[1], reverse=True)[1:num_subs_to_reccomend+1]
    recommend_frame = []
    for val in closest_subs:
        recommend_frame.append({'Subreddit':val[0],'Similarity':val[1].item(0)})

    df = pd.DataFrame(recommend_frame)
    df = df.set_index(['Subreddit'])
    return df

In [ ]:
df_control = get_similar_subreddits("CryptoCurrencies", 50)
df_control

,Similarity
Subreddit,
ethfinance,0.697069
eos,0.674924
cardano,0.672937
binance,0.646476
CryptoCurrency,0.642692
SPCE,0.623980
TREZOR,0.621438
ethtrader,0.621283
Tronix,0.616831


## Calculating the Diversity

The diversity of a set of items, c1,...cn, is defined as the average dissimilarity between all pairs of items in the result set.
{add equation}

Here, we can calculate the average dissimilarity of items recommended when no diversity algorithms are implemented. This will be used as a control to help us evaluate and compare our results later on.

In [ ]:
n = 50
dis_similarity = [x for x in pdist(df_control)]

avg_dissim_control = (sum(dis_similarity))/((n/2)*(n-1))
avg_dissim_control

0.03973605662801884

## Bounded Greedy Selection Algorithm

The Greedy Selection algorithm seeks to provide a more principled approach to improving diversity by using a quality metric to guide the construction of a result set, R, in an incremental fashion. During each iteration the remaining items are ordered according to their quality and the highest quality item added to R. The first item to be selected is always the one with the highest similarity to the target. During each subsequent iteration, the item selected is the one with the highest quality with respect to the set of cases selected during the previous iteration. This algorithm is expensive.

To reduce the complexity of the Greedy Selection algorithm we can implement a bounded version. The Bounded Greedy Selection algorithm first selects the best x cases according to their similarity to the target query and then applies the greedy selection method to these. 

[Source](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.8.5232&rep=rep1&type=pdf)

### Step 1: Select the best x = 500 cases according to their similarity to the target query. Set C'

In [ ]:
def similar_subreddits(target, num_subs_to_reccomend):
    similarities = []
    sub_name_vector = dfnew['vector'][dfnew['Labels'] == target].to_numpy()[0]
    sub_name_vector_reshaped = np.array(sub_name_vector).reshape(1, -1)
    for vector in dfnew['vector'].tolist():
        vector_reshaped = np.array(vector).reshape(1, -1)
        similarities.append(cosine_similarity(sub_name_vector_reshaped, vector_reshaped))

    pairs = list(zip(dfnew['Labels'], similarities, dfnew['vector']))
    closest_subs = sorted(pairs, key=lambda item: item[1], reverse=True)[1:num_subs_to_reccomend+1]
    recommend_frame = []
    for val in closest_subs:
        recommend_frame.append({'Subreddit':val[0],'Similarity':val[1].item(0), 'Vector':val[2]})

    df = pd.DataFrame(recommend_frame)
    return df

In [ ]:
C_prime = similar_subreddits("CryptoCurrencies", 500)
C_prime

,Subreddit,Similarity,Vector
0,ethfinance,0.697069,"[0.15285654, -0.08509623, -0.15935297, -0.1228..."
1,eos,0.674924,"[-0.036491897, -0.083486974, -0.04064292, -0.2..."
2,cardano,0.672937,"[-0.15176040000000002, -0.34675809999999996, 0..."
3,binance,0.646476,"[0.069142476, -0.16032465, -0.31838068, -0.217..."
4,CryptoCurrency,0.642692,"[0.25012629999999997, 0.022930225, 0.31550872,..."
...,...,...,...
495,FlareNetworks,0.428905,"[-0.039291855, -0.29546592, 0.1205579040000000..."
496,bsv,0.428890,"[0.14520642, -0.087743, 0.00958406, -0.2112380..."
497,pennystonkbets,0.428804,"[0.25879478, -0.20589523, 0.23740718, -0.20989..."
498,Citibike,0.428767,"[0.2993206, -0.24152857, 0.27609712, -0.290765..."


### Step 2: Add the most similar item from C' as the first item in the result set R and drop this item from C'

In [ ]:
df_temp = C_prime
recommendations = ['dummy']
recommendations[0] = C_prime["Subreddit"][0]  # first item is always the one with the highest similarity

index = df_temp[(df_temp.Subreddit == recommendations[0])].index

df_temp = df_temp.drop(index)

### Step 3: During each subsequent iteration, the item selected is the one with the highest quality with respect to the set of cases selected during the previous iteration

The quality of an item c is proportional to the similarity between c and the current target t, and to the diversity of c relative to those items so far selected, R = {r1,...,rm}.

Quality(t,c,R) = Similarity(t,c) ∗ RelDiversity(c,R)

In [ ]:
def calculate_quality(c, R, df, df_sim):
    quality = 0
    rel_diversity = 0
    
    if len(R) == 0:
        rel_diversity = 1
        
    vector = np.array(df['Vector'][df['Subreddit'] == c].to_numpy()[0]).reshape(1, -1)
    diversity = []
    for item in R:
        diversity.append(1 - cosine_similarity(vector, np.array(df_sim['Vector'][df_sim['Subreddit'] == item].to_numpy()[0]).reshape(1, -1)))
        
    rel_diversity = sum(diversity)/len(R) # relative diversity
    
    similarity = df['Similarity'][df['Subreddit'] == c].to_numpy()[0] # similarity
    
    quality = rel_diversity[0][0] * similarity # quality
    return quality

In [ ]:
# set k = 50 to get top 50 recommendations
k = 50
for i in range(k):
    qualities = {}
    # Calculate the quality of each subreddit
    for item in df_temp['Subreddit']:
        qualities[item] = calculate_quality(item, recommendations, df_temp, C_prime)

    highest_quality = max(qualities.values())
    highest_quality_subreddit = max(qualities, key= lambda x: qualities[x])
    recommendations.append(highest_quality_subreddit)
    
    index = df_temp[(df_temp.Subreddit == recommendations[-1])].index
    df_temp = df_temp.drop(index)

In [ ]:
recommendations

['ethfinance',
 'EnemyTerritory',
 'ADMP',
 'CryptoCurrencyMeta',
 'dogecoindev',
 'AltStreetBets',
 'zec',
 'altcoin',
 'canoo',
 'barinsta',
 'TREZOR',
 'CardanoMarkets',
 'CryptoCurrency',
 'HighTideInc',
 'binance',
 'eos',
 'kucoin',
 'cardano',
 'SPCE',
 'EnergyWebToken',
 'AlgorandOfficial',
 'NFT',
 'cryptocurrencymemes',
 'romanianrichdad',
 'CardanoStakePools',
 'Buttcoin',
 'LINKTrader',
 'CelsiusNetwork',
 'Pennystock',
 'Tronix',
 'Dogecoinfah',
 'xlm',
 'ethtrader',
 'UKInvesting',
 'loopringorg',
 'Network',
 'tezos',
 'NervosNetwork',
 'Vechain',
 'ChurchillCapital',
 'zorinos',
 'CryptoHorde',
 'Stellar',
 'banano',
 'CardanoTrading',
 'PennyStocksDD',
 'ArkEcosystem',
 'Chainlink',
 'daonuts',
 'waltonchain',
 'Cannabisstockmarket']

## Evaluate the Recommendations

In [ ]:
similarities = []
for item in recommendations:
    sim = C_prime['Similarity'][C_prime['Subreddit'] == item].to_numpy()[0]
    similarities.append(sim)

pairs = list(zip(recommendations, similarities))
recommend_frame = []
for val in pairs:
    recommend_frame.append({'Subreddit':val[0],'Similarity':val[1].item(0)})    

df_sim = pd.DataFrame(recommend_frame)
df_sim = df_sim.set_index(['Subreddit'])
df_sim

,Similarity
Subreddit,
ethfinance,0.697069
EnemyTerritory,0.467882
ADMP,0.451738
CryptoCurrencyMeta,0.585481
dogecoindev,0.583678
AltStreetBets,0.613051
zec,0.574186
altcoin,0.599149
canoo,0.579816


In [ ]:
# Find the Diversity
n = 50
dis_similarity = [x for x in pdist(df_sim)]
avg_dissim_greedy = (sum(dis_similarity))/((n/2)*(n-1))
avg_dissim_greedy

0.07724201777766121

## Compare Results to Normal RecSys

We can compare the average dissimilarity of these new, diverse recommendations to our original ones in order to compare and evaluate the diversity in each set.

In [ ]:
percent_change = ((avg_dissim_greedy - avg_dissim_control)/avg_dissim_control)*100

In [ ]:
round(percent_change, 2)

94.39

Thus, there was a 94.4% increase in diversity (defined as average dissimilarity) when we move from a normal reccomendation system to a system that implements the Bounded Greedy Selection algorithm.